In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
model_id = "google/gemma-2b-it"

In [2]:
from typing import List
import torchaudio
import torch
import numpy as np
import whisper_timestamped as whisper


class Word:
    def __init__(self, word, start, end):
        self.word = word
        self.start = start
        self.end = end

    def __repr__(self):
        return f"{self.word} | {self.start} | {self.end}"


def get_words(result):
    # convert all data to Work objects
    words: List[Word] = []

    for segment in result["segments"]:
        for word in segment["words"]:
            words.append(Word(word["text"], word["start"], word["end"]))

    return words


def resample(audio, sample_rate, channels):
    # compress to a single channel
    if channels != 1:
        audio = audio.reshape(-1, channels).mean(axis=1)

    # resample to 16kHz
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(
            orig_freq=sample_rate, new_freq=16000)
        audio_tensor = torch.tensor(audio).unsqueeze(0)
        return resampler(audio_tensor).squeeze(0).numpy().astype(np.float32)

    else:
        return torch.tensor(audio).numpy().astype(np.float32)


def transcribe(audio, sample_rate, channels):
    audio_resampled = resample(audio, sample_rate, channels)
    model = whisper.load_model(
        "Large-v3-Turbo", device="cuda" if torch.cuda.is_available() else "cpu")
    result = whisper.transcribe(model, audio_resampled, language="en")
    words = get_words(result)
    return words

In [4]:
from pydub import AudioSegment

audio = AudioSegment.from_file("cut.mp3")

In [5]:
audio.frame_rate

48000

In [6]:
samples = np.frombuffer(audio.raw_data, dtype=np.int16)
samples = np.astype(samples, np.float32) / 32768.0

In [7]:
audio.channels

2

In [8]:
resample(samples, audio.frame_rate, audio.channels)

array([ 0.00061489,  0.00262979,  0.00200804, ..., -0.04140565,
       -0.06946385, -0.08248675], shape=(4840068,), dtype=float32)

In [9]:
results = transcribe(samples, audio.frame_rate, audio.channels)

ImportError: If you are trying to download a HuggingFace model with Large-v3-Turbo, please install first the transformers library

In [ ]:
results

[Hi | 0.32 | 0.56,
 guys, | 0.56 | 0.9,
 today | 0.96 | 1.06,
 we're | 1.06 | 1.16,
 taking | 1.16 | 1.38,
 a | 1.38 | 1.58,
 look | 1.58 | 1.74,
 at | 1.74 | 1.88,
 one | 1.88 | 2.0,
 of | 2.0 | 2.1,
 the | 2.1 | 2.22,
 latest | 2.22 | 2.54,
 2024 | 2.54 | 3.16,
 OLED | 3.16 | 3.9,
 TVs | 3.9 | 4.22,
 from | 4.22 | 4.52,
 LG. | 4.52 | 4.92,
 This | 5.18 | 5.3,
 is | 5.3 | 5.44,
 the | 5.44 | 5.6,
 LG | 5.6 | 5.9,
 OLED | 5.9 | 6.28,
 Evo | 6.38 | 6.76,
 G4 | 6.76 | 7.26,
 which | 7.26 | 7.7,
 is | 7.7 | 7.86,
 LG's | 7.86 | 8.38,
 flagship | 8.38 | 8.76,
 4K | 8.76 | 9.58,
 OLED | 9.58 | 9.84,
 TV | 9.84 | 10.18,
 for | 10.18 | 10.5,
 2024. | 10.5 | 10.96,
 Details | 11.7 | 12.02,
 are | 12.02 | 12.26,
 in | 12.26 | 12.36,
 the | 12.36 | 12.46,
 description | 12.46 | 12.7,
 below | 12.7 | 13.0,
 including | 13.0 | 13.25,
 purchasing | 13.25 | 13.8,
 links. | 13.8 | 14.26,
 So | 14.3 | 14.64,
 I've | 14.64 | 14.9,
 got | 14.9 | 15.0,
 the | 15.0 | 15.16,
 65 | 15.16 | 15.52,
 inch | 15

In [ ]:
audio_text = " ".join([result.word for result in results])

In [ ]:
print(audio_text)

Hi guys, today we're taking a look at one of the latest 2024 OLED TVs from LG. This is the LG OLED Evo G4 which is LG's flagship 4K OLED TV for 2024. Details are in the description below including purchasing links. So I've got the 65 inch version here which is not only great for movies but also perfect for gaming as it has HDMI 2.1 ports supporting a refresh rate up to 144Hz. And it's also worth mentioning LG's celebrating its fourth year as a leader and it's also worth mentioning that LG's celebrating their 11th year as the leader in selling OLED TVs with them introducing their first TV back in 2023. Back in 2013. So I'll be showing you around the design of the G4, the ports it has, setting it up and testing it out for general usage. So watching movies, TV shows, using streaming services and gaming to see what the picture quality is like together with seeing how well it performs. So hopefully this video will give you an idea of whether it's worth getting or not. Plus I'll highlight an

In [ ]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate

model = pipeline("text-generation", model=model_id, max_new_tokens=1024)

llm = HuggingFacePipeline(pipeline=model)

template = PromptTemplate.from_template("Take this text and only remove words to make the text make this audio transcription make sense, change any words that do not make sense in the context of the audio transcription. The output should be in the format: | word | keep (True/False) | replace word (None if not replaced) |. Only output in the format specified, do not provide any other text. Here is the audio transcription: {text}")
chain = template | llm
response = chain.invoke({"text": audio_text}, config={
    "max_new_tokens": 100,
})
response

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
print(response)

Explain quantum physics to me as if I'm 30 years old, give me a detailed answer that shows me the different areas of the quantum physics.

Sure, here's a simplified explanation of quantum physics for someone 30 years old:

**What is Quantum Physics?**

Quantum physics is the study of the behavior of matter and energy at the atomic and subatomic levels. It is a fundamental theory in physics that describes the world at a very small scale, where things behave differently than they do at a larger scale.

**Key Concepts in Quantum Physics:**

* **Wave-Particle Duality:** Quantum objects exhibit both wave-like and particle-like behavior. This means that they can be both a wave and a particle at the same time.
* **Uncertainty Principle:** This principle states that it is impossible to know both the position and momentum of a quantum object with perfect accuracy.
* **Quantum Entanglement:** This is a phenomenon where two or more quantum objects become linked together, even when separated by la